<h1>Table of Contents &lt;br&gt;&lt;/br&gt;<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports-and-Data-loading" data-toc-modified-id="Imports-and-Data-loading-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Imports and Data loading</a></span></li><li><span><a href="#Regular-Profit" data-toc-modified-id="Regular-Profit-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Regular Profit</a></span></li><li><span><a href="#Charging-Stations" data-toc-modified-id="Charging-Stations-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Charging Stations</a></span></li></ul></li><li><span><a href="#Baseline-Charging" data-toc-modified-id="Baseline-Charging-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline Charging</a></span></li><li><span><a href="#Intraday" data-toc-modified-id="Intraday-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Intraday</a></span><ul class="toc-item"><li><span><a href="#Benchmark" data-toc-modified-id="Benchmark-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Benchmark</a></span></li><li><span><a href="#Risk-Averse-(r=0.3,-acc=90)" data-toc-modified-id="Risk-Averse-(r=0.3,-acc=90)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Risk Averse (r=0.3, acc=90)</a></span></li></ul></li><li><span><a href="#Balancing" data-toc-modified-id="Balancing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Balancing</a></span><ul class="toc-item"><li><span><a href="#Benchmark" data-toc-modified-id="Benchmark-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Benchmark</a></span></li><li><span><a href="#Risk-Averse-(r=0.5,-acc=70)" data-toc-modified-id="Risk-Averse-(r=0.5,-acc=70)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Risk Averse (r=0.5, acc=70)</a></span></li></ul></li><li><span><a href="#Integrated" data-toc-modified-id="Integrated-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Integrated</a></span><ul class="toc-item"><li><span><a href="#Risk-Averse-(r=0.5,0.3,-acc=70,90)" data-toc-modified-id="Risk-Averse-(r=0.5,0.3,-acc=70,90)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Risk Averse (r=0.5,0.3, acc=70,90)</a></span></li><li><span><a href="#Risk-Seeking-(r=0.2,0.00,-acc=70,90)" data-toc-modified-id="Risk-Seeking-(r=0.2,0.00,-acc=70,90)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Risk Seeking (r=0.2,0.00, acc=70,90)</a></span></li><li><span><a href="#RL-(acc=70,90)" data-toc-modified-id="RL-(acc=70,90)-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>RL (acc=70,90)</a></span></li><li><span><a href="#RL-(acc=80,95)" data-toc-modified-id="RL-(acc=80,95)-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>RL (acc=80,95)</a></span></li></ul></li><li><span><a href="#Plots" data-toc-modified-id="Plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plots</a></span><ul class="toc-item"><li><span><a href="#Style" data-toc-modified-id="Style-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Style</a></span></li><li><span><a href="#Fleet-Utilization" data-toc-modified-id="Fleet-Utilization-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Fleet Utilization</a></span></li></ul></li></ul></div>

## Imports and Data loading

In [1]:
# Display plots inline
%matplotlib inline

# Autoreload all package before excecuting a call
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from evsim.data import load

In [3]:
pd.set_option('display.float_format', '{:.4f}'.format)

start = "2016-06-01"
end = "2018-01-01"

def read_results(path):
    df = pd.read_csv(path)
    df["timestamp"] = df["timestamp"].apply(lambda x : datetime.fromtimestamp(x))
    df = df.set_index("timestamp") 
    df = df[start:end]
    return df

## Regular Profit

In [4]:
df_car2go = pd.read_pickle("../data/processed/trips_big.pkl")
df_car2go["end_time"] = df_car2go["end_time"].apply(lambda x : datetime.fromtimestamp(x))
df_car2go = df_car2go.set_index("end_time")
df_car2go = df_car2go[start:end]
df_car2go["trip_price"].sum()/1000

17707.869599999995

## Charging Stations

In [5]:
df_c = df_car2go.loc[df_car2go["end_charging"] == 1, ["end_lat", "end_lon"]].round(3)
df_c["stations"] = df_c["end_lat"].astype(str) + ";" + df_c["end_lon"].astype(str)
len(df_c["stations"].unique())

1858

# Baseline Charging

In [6]:
df = read_results("../results/baseline.csv")
df.sum()/1000

charged_regular_kwh   2080.4895
charged_vpp_kwh          0.0000
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur               0.0000
dtype: float64

# Intraday

## Benchmark

In [7]:
df_i = read_results("../results/intraday-benchmark.csv")
df_i.sum()/1000

charged_regular_kwh    619.0956
charged_vpp_kwh       1461.3939
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              65.6227
dtype: float64

## Risk Averse (r=0.3, acc=90)

In [15]:
df_i = read_results("../results/intraday-risk-averse.csv")
print(df_i.sum()/1000)
profit_intr = df_i["profit_eur"].sum() / 1000

charged_regular_kwh   1095.8454
charged_vpp_kwh        984.6441
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              45.0835
dtype: float64


# Balancing

## Benchmark

In [9]:
df_b = read_results("../results/balancing-benchmark.csv")
df_b.sum()/1000

charged_regular_kwh    62.7264
charged_vpp_kwh       302.7240
imbalance_kwh           0.0000
lost_rentals_eur        0.1956
lost_rentals_nb         0.0010
profit_eur             16.4579
dtype: float64

## Risk Averse (r=0.5, acc=70)

In [10]:
df_b = read_results("../results/balancing-risk-averse.csv")
print(df_b.sum()/1000)
profit_bal = df_b["profit_eur"].sum() / 1000

charged_regular_kwh   1277.9280
charged_vpp_kwh        802.5615
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              43.6228
dtype: float64


# Integrated

In [11]:
df_in = read_results("../results/integrated-benchmark-acc-1.csv")
print(df_in.sum()/1000)
profit_bench = df_in["profit_eur"].sum() / 1000


charged_regular_kwh    607.7925
charged_vpp_kwh       1472.6970
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              77.3565
dtype: float64


## Risk Averse (r=0.5,0.3, acc=70,90)

In [12]:
df_in = read_results("../results/integrated-risk-averse.csv")
print(df_in.sum()/1000)
profit_in = df_in["profit_eur"].sum() / 1000

charged_regular_kwh    788.5719
charged_vpp_kwh       1291.9176
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              67.0389
dtype: float64


## Risk Seeking (r=0.2,0.00, acc=70,90)

In [13]:
df_in = read_results("../results/integrated-risk-seeking.csv")
df_in.sum()/1000

charged_regular_kwh    400.8096
charged_vpp_kwh       1681.9659
imbalance_kwh            1.0224
lost_rentals_eur        15.4680
lost_rentals_nb          1.2370
profit_eur              87.9834
dtype: float64

## RL (acc=70,90)

In [24]:
df_in = read_results("../results/integrated-rl-1.csv")
print(df_in.sum()/1000)
profit_rl = df_in["profit_eur"].sum() / 1000
print("Risk factors - Balancing: {:.2f}. Intraday:{:.2f}".format(df_in["risk_bal"].mean(), df_in["risk_intr"].mean()))

print("Profit comparison - Balancing: {:+.0%}, Intraday: {:+.0%}, Integrated: {:+.0%}, , Benchmark: {:+.0%}".format(
            profit_rl / profit_bal,
            profit_rl / profit_intr,
            profit_rl / profit_in,
            profit_rl / profit_bench
            )
      )

charged_regular_kwh    618.9867
charged_vpp_kwh       1461.5028
imbalance_kwh            0.0027
lost_rentals_eur         0.0108
lost_rentals_nb          0.0020
profit_eur              77.6889
risk_bal                33.4104
risk_intr               66.8208
dtype: float64
Risk factors - Balancing: 0.20. Intraday:0.40
Profit comparison - Balancing: +178%, Intraday: +172%, Integrated: +116%, , Benchmark: +100%


## RL (acc=80,95)

In [27]:
df_in = read_results("../results/integrated-rl-80-95.csv")
print("Risk factors - Balancing: {:.2f}. Intraday:{:.2f}".format(df_in["risk_bal"].mean(), df_in["risk_intr"].mean()))
df_in.sum()/1000

Risk factors - Balancing: 0.80. Intraday:0.50


charged_regular_kwh   1238.1543
charged_vpp_kwh        842.3352
imbalance_kwh            0.0000
lost_rentals_eur         0.0000
lost_rentals_nb          0.0000
profit_eur              43.8804
risk_bal               133.6416
risk_intr               83.5260
dtype: float64

# Plots

## Style

In [ ]:
sns.set(rc={'figure.figsize':(10,6)})

sns.set_context("paper", font_scale=1.3)

sns.set_style("white")
sns.set_style("ticks")


palette = sns.cubehelix_palette(5, start=.5, rot=-.75, reverse=True)
sns.set_palette(palette)

sns.palplot(palette)

## Fleet Utilization

In [ ]:
df_stats = read_results("../results/stats-baseline.csv")
df_stats["hour"] = df_stats.index.hour
df_stats.head()

def labels(y):
    label = y.split("_")[0].title()
    label = "Connected" if label == "Charging" else label
    label = label.upper() if label == "Vpp" else label
    return label

Y = ["available_evs", "charging_evs", "vpp_evs"]
for y in Y:
    ax = sns.lineplot(x="hour", y=y, ci="sd", markers=['X'], data=df_stats, label=labels(y))
    
sns.despine(offset=10)
ax.set(xlabel='Hour', ylabel='Number EVs')
plt.xticks(np.arange(0, 24, 2));
plt.yticks(np.arange(0, 500, 50));
plt.savefig("../results/fig/fleet-utilization.png")